# MNIST uDNN Model Pre-training
Credit: this notebook is based on Keras official documents https://keras.io/examples/vision/mnist_convnet/

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1-Bn1Z7XItLokDJanKX9I3KlBWWjXI-RE?usp=sharing)

In [20]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt

## Use Google Colab Pro GPU (Optional)

In [21]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Import MNIST Dataset

In [22]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(trainX, trainY), (testX, testY) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
trainX = trainX.astype("float32") / 255
testX = testX.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
trainX = np.expand_dims(trainX, -1)
testX = np.expand_dims(testX, -1)
print("trainX shape:", trainX.shape)
print(trainX.shape[0], "train samples")
print(testX.shape[0], "test samples")


# convert class vectors to binary class matrices
trainY = keras.utils.to_categorical(trainY, num_classes)
testY = keras.utils.to_categorical(testY, num_classes)

trainX shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Define uDNN Model

In [23]:
model = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(16, kernel_size=(3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(32, kernel_size=(3,3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

## Compile and Train Model

In [24]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
model.fit(trainX, trainY, epochs=15, batch_size=128, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 2s 5ms/step - loss: 0.5084 - accuracy: 0.8396 - val_loss: 0.1133 - val_accuracy: 0.9695
Epoch 2/15
422/422 [==============================] - 2s 4ms/step - loss: 0.1691 - accuracy: 0.9494 - val_loss: 0.0736 - val_accuracy: 0.9790
Epoch 3/15
422/422 [==============================] - 2s 4ms/step - loss: 0.1277 - accuracy: 0.9610 - val_loss: 0.0636 - val_accuracy: 0.9835
Epoch 4/15
422/422 [==============================] - 2s 4ms/step - loss: 0.1067 - accuracy: 0.9679 - val_loss: 0.0542 - val_accuracy: 0.9870
Epoch 5/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0965 - accuracy: 0.9707 - val_loss: 0.0475 - val_accuracy: 0.9867
Epoch 6/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0867 - accuracy: 0.9736 - val_loss: 0.0426 - val_accuracy: 0.9885
Epoch 7/15
422/422 [==============================] - 2s 4ms/step - loss: 0.0810 - accuracy: 0.9749 - val_loss: 0.0423 - val_accuracy: 0.9888
Epoch 

In [26]:
model.evaluate(testX, testY)

313/313 [==============================] - 1s 2ms/step - loss: 0.0305 - accuracy: 0.9901


[0.030508821830153465, 0.9901000261306763]

## Prepare to Export Model

In [27]:
! git clone https://github.com/leleonardzhang/uDNN-tf2msp.git
! pip install fxpmath
import sys
from fxpmath import Fxp
sys.path.insert(0, '/content/uDNN-tf2msp')
import encoder

fatal: destination path 'uDNN-tf2msp' already exists and is not an empty directory.


## Export Model to Header File

In [28]:
encoder.export_model(model)

## Get Sample Input and Output (Optional)

In [29]:
test_n = 1      # the nth data in test dataset
print("Fixed Point Input")
print(Fxp(testX[test_n].transpose(2,0,1), signed = True, n_word = 16, n_frac = 10).val.flatten().tolist())
print("Output Label")
print(np.argmax(testY[test_n]))

Fixed Point Input
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 465, 501, 686, 1024, 1024, 602, 373, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 678, 1015, 1015, 1015, 1015, 1015, 1015, 875, 120, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 678, 1015, 1015, 1015, 855, 570, 706, 1015, 1015, 489, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 208, 1003, 1015, 843, 128, 48, 0, 24, 827, 1015, 562, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 309, 1007, 843, 100, 0, 0, 0, 489, 995, 1015, 261, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 124, 72, 0, 0, 0, 0, 839, 1015, 1015, 261, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 469, 991, 1015, 795, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Next Step

1.   Download header file `neural_network_parameters.h` and copy the header file into the root directory of MSP implementation `uDNN-tf2msp-msp_impl/`.
2.   Modify the pointer `input_buffer` in `neural_network_parameters.h` and point to desired data input.
3.   Compile and run the program on MSP.
